In [ ]:

!pip install pythainlp huggingface_hub sentencepiece
!pip install torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu121
!pip install fastai==2.7.13 transformers==4.41.0 datasets rouge_score
!pip install --force-reinstall numpy==1.24.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 30.8 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 719.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torchvision-0.21.0+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [ ]:
import pandas as pd
import numpy as np
from pythainlp.tokenize import word_tokenize
from fastai.text.all import *
from rouge_score import rouge_scorer
import torch
import warnings
import pickle
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset

warnings.filterwarnings('ignore')

### 1. DATA LOADING & VALIDATION ###############################################
def load_and_validate_data(csv_path):
    """Load data with strict validation"""
    df = pd.read_csv(csv_path, encoding='utf-8-sig')

    # Validate structure
    assert {'text', 'summary'}.issubset(df.columns), "Missing required columns"

    # Clean data
    df = df.dropna(subset=['text', 'summary'])
    df = df[(df['text'].str.len() > 30) & (df['summary'].str.len() > 15)]
    df = df.reset_index(drop=True)
    print(f"Loaded {len(df)} documents")

    # Formal language checks
    informal_markers = ['ครับ', 'ค่ะ', 'นะ']
    for marker in informal_markers:
        if df['summary'].str.contains(marker).any():
            print(f"Warning: Informal marker '{marker}' detected")

    print(f"Loaded {len(df)} clean pairs")
    return df

# Load data
csv_path = "formal_thai.csv"
df = load_and_validate_data(csv_path)

### 2. CUSTOM TOKENIZER ########################################################
class ThaiTokenizer:
    def __init__(self):
        self.special_toks = ['xxbos', 'xxeos', 'xxpad', 'xxunk']
        self.formal_map = {'ครับ': 'คะ', 'ค่ะ': 'คะ', 'นะครับ': 'คะ'}

    def __call__(self, text, **kwargs):
        tokens = word_tokenize(str(text), engine="newmm")
        tokens = [self.formal_map.get(t, t) for t in tokens if t.strip()]
        return ['xxbos'] + tokens + ['xxeos']

tokenizer = ThaiTokenizer()

# Test tokenizer
sample_text = "เอกสารนี้ระบุว่า ตามที่ได้หารือกันครับ"
print("Tokenized example:", tokenizer(sample_text))

### 3. DATA PREPARATION & DATALOADER ##########################################
def prepare_dataloaders(df, seq_len=64, bs=4):
    print("Tokenizing texts...")
    sources = [tokenizer(str(t)) for t in df['text']]
    targets = [tokenizer(str(t)) for t in df['summary']]

    print("Creating vocabulary...")
    all_tokens = [tok for sublist in sources+targets for tok in sublist]
    counter = Counter(all_tokens)
    vocab = ['xxbos', 'xxeos', 'xxpad', 'xxunk'] + [tok for tok, cnt in counter.most_common(30000)]
    word2idx = {word: i for i, word in enumerate(vocab)}
    pad_idx = word2idx['xxpad']
    unk_idx = word2idx['xxunk']

    # Save vocabulary
    with open("thai_vocab.pkl", "wb") as f:
        pickle.dump(word2idx, f)

    # Convert tokens to indices with fallback to xxunk
    def to_idx(seq, word2idx):
        return [word2idx.get(w, unk_idx) for w in seq]

    def pad_or_trim(t, length, pad_idx):
        if len(t) < length:
            return F.pad(t, (0, length - len(t)), value=pad_idx)
        else:
            return t[:length]

    src_nums = [pad_or_trim(torch.tensor(to_idx(seq, word2idx)), seq_len, pad_idx) for seq in sources]
    tgt_nums = [pad_or_trim(torch.tensor(to_idx(seq, word2idx)), seq_len, pad_idx) for seq in targets]

    # Create Datasets and DataLoaders
    padded_src = torch.stack(src_nums)
    padded_tgt = torch.stack(tgt_nums)
    train_ds = TensorDataset(padded_src, padded_tgt)

    dls = DataLoaders.from_dsets(
        train_ds,
        valid_pct=0.2,
        bs=bs,
        shuffle=True
    )

    print("\nSuccess! DataLoaders created with:")
    print(f"- Vocab size: {len(vocab)}")
    print(f"- Input shape: {padded_src[0].shape}")
    print(f"- Target shape: {padded_tgt[0].shape}")
    return dls

# Create and verify DataLoaders
dls = prepare_dataloaders(df)

# Verify one batch
x, y = dls.one_batch()
print(f"Input shape: {x.shape}")
print(f"Target shape: {y.shape}")

Loaded 50 documents
Loaded 50 clean pairs
Tokenized example: ['xxbos', 'เอกสาร', 'นี้', 'ระบุ', 'ว่า', 'ตามที่', 'ได้', 'หารือ', 'กัน', 'คะ', 'xxeos']
Tokenizing texts...
Creating vocabulary...

Success! DataLoaders created with:
- Vocab size: 2067
- Input shape: torch.Size([64])
- Target shape: torch.Size([64])
Input shape: torch.Size([4, 64])
Target shape: torch.Size([4, 64])


In [ ]:
import pandas as pd
import numpy as np
from pythainlp.tokenize import word_tokenize
from fastai.text.all import *
from rouge_score import rouge_scorer
import torch
import torch.nn as nn
import torch.nn.functional as F
import warnings
import pickle
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader

warnings.filterwarnings('ignore')

### 1. LOAD & CLEAN DATA ##################################################
def load_and_validate_data(csv_path):
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    assert {'text', 'summary'}.issubset(df.columns), "Missing required columns"
    df = df.dropna(subset=['text', 'summary'])
    df = df[(df['text'].str.len() > 30) & (df['summary'].str.len() > 15)]
    df = df.reset_index(drop=True)

    # Check for informal tone
    informal_markers = ['ครับ', 'ค่ะ', 'นะ']
    for marker in informal_markers:
        if df['summary'].str.contains(marker).any():
            print(f"Warning: Informal marker '{marker}' detected in summaries")

    print(f"Loaded {len(df)} clean pairs")
    return df

csv_path = "formal_thai.csv"
df = load_and_validate_data(csv_path)

### 2. THAI TOKENIZER ######################################################
class ThaiTokenizer:
    def __init__(self):
        self.special_toks = ['xxbos', 'xxeos', 'xxpad', 'xxunk']
        self.formal_map = {'ครับ': 'คะ', 'ค่ะ': 'คะ', 'นะครับ': 'คะ'}

    def __call__(self, text, **kwargs):
        tokens = word_tokenize(str(text), engine="newmm")
        tokens = [self.formal_map.get(t, t) for t in tokens if t.strip()]
        return ['xxbos'] + tokens + ['xxeos']

tokenizer = ThaiTokenizer()

### 3. DATASET & DATALOADER PREP ############################################
def prepare_dataloaders(df, seq_len=64, bs=4):
    print("Tokenizing texts...")
    sources = [tokenizer(str(t)) for t in df['text']]
    targets = [tokenizer(str(t)) for t in df['summary']]

    print("Creating vocabulary...")
    all_tokens = [tok for sublist in sources+targets for tok in sublist]
    counter = Counter(all_tokens)
    vocab = ['xxbos', 'xxeos', 'xxpad', 'xxunk'] + [tok for tok, cnt in counter.most_common(30000)]
    word2idx = {word: i for i, word in enumerate(vocab)}
    idx2word = {i: w for w, i in word2idx.items()}
    pad_idx = word2idx['xxpad']
    unk_idx = word2idx['xxunk']

    # Save vocab
    with open("thai_vocab.pkl", "wb") as f:
        pickle.dump(word2idx, f)

    def to_idx(seq): return [word2idx.get(w, unk_idx) for w in seq]
    def pad_or_trim(t, length, pad_idx):
        if len(t) < length:
            return F.pad(t, (0, length - len(t)), value=pad_idx)
        else:
            return t[:length]

    src_nums = [pad_or_trim(torch.tensor(to_idx(seq)), seq_len, pad_idx) for seq in sources]
    tgt_nums = [pad_or_trim(torch.tensor(to_idx(seq)), seq_len, pad_idx) for seq in targets]

    padded_src = torch.stack(src_nums)
    padded_tgt = torch.stack(tgt_nums)

    # Manual split to avoid 0 validation set
    split_idx = int(0.8 * len(padded_src))
    train_ds = TensorDataset(padded_src[:split_idx], padded_tgt[:split_idx])
    valid_ds = TensorDataset(padded_src[split_idx:], padded_tgt[split_idx:])

    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
    valid_dl = DataLoader(valid_ds, batch_size=bs)

    dls = DataLoaders(train_dl, valid_dl)

    print("\n DataLoaders created!")
    print(f"- Vocab size: {len(vocab)}")
    print(f"- Input shape: {padded_src[0].shape}")
    return dls, vocab, pad_idx

dls, vocab, pad_idx = prepare_dataloaders(df)


### 4. MODEL DEFINITION (LSTM Encoder-Decoder) #############################
class Thai2FitSummarizer(nn.Module):
    def __init__(self, vocab_size, pad_idx, emb_sz=300, hidden_sz=512, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_sz, padding_idx=pad_idx)
        self.encoder = nn.LSTM(emb_sz, hidden_sz, num_layers, batch_first=True)
        self.decoder = nn.LSTM(emb_sz, hidden_sz, num_layers, batch_first=True)
        self.out = nn.Linear(hidden_sz, vocab_size)

    def forward(self, src, tgt):
        embedded_src = self.embedding(src)
        _, (hidden, cell) = self.encoder(embedded_src)

        embedded_tgt = self.embedding(tgt)
        output, _ = self.decoder(embedded_tgt, (hidden, cell))

        logits = self.out(output)
        return logits

# Instantiate model
model = Thai2FitSummarizer(vocab_size=len(vocab), pad_idx=pad_idx)
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_idx)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)


### 5. TRAINING LOOP #########################################################
def train(model, dls, loss_fn, opt, epochs=5, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for xb, yb in dls.train:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            out = model(xb, yb[:, :-1])
            loss = loss_fn(out.reshape(-1, out.shape[-1]), yb[:, 1:].reshape(-1))
            loss.backward()
            opt.step()
            total_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for xb, yb in dls.valid:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb, yb[:, :-1])
                loss = loss_fn(out.reshape(-1, out.shape[-1]), yb[:, 1:].reshape(-1))
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs} — Train Loss: {total_loss:.4f}, Val Loss: {val_loss:.4f}")

# Train the model
train(model, dls, loss_fn, opt, epochs=10)



Loaded 50 clean pairs
Tokenizing texts...
Creating vocabulary...

 DataLoaders created!
- Vocab size: 2067
- Input shape: torch.Size([64])
Epoch 1/10 — Train Loss: 71.4488, Val Loss: 19.0999
Epoch 2/10 — Train Loss: 60.7967, Val Loss: 19.2435
Epoch 3/10 — Train Loss: 56.9685, Val Loss: 18.9287
Epoch 4/10 — Train Loss: 53.4582, Val Loss: 18.1939
Epoch 5/10 — Train Loss: 49.8175, Val Loss: 17.7863
Epoch 6/10 — Train Loss: 45.9788, Val Loss: 17.0412
Epoch 7/10 — Train Loss: 41.6886, Val Loss: 16.8431
Epoch 8/10 — Train Loss: 37.5046, Val Loss: 16.5128
Epoch 9/10 — Train Loss: 33.2811, Val Loss: 16.2750
Epoch 10/10 — Train Loss: 29.1387, Val Loss: 16.0510


In [ ]:
def idx_to_text(idx_list, idx2word):
    return ' '.join([idx2word.get(idx, 'xxunk') for idx in idx_list if idx != pad_idx and idx2word.get(idx) not in ['xxbos', 'xxpad']])

def generate_summary(input_text, model, tokenizer, word2idx, idx2word, max_len=64, device='cpu'):
    model.eval()
    model.to(device)

    # Prepare input sequence
    tokens = tokenizer(input_text)
    input_idxs = [word2idx.get(tok, word2idx['xxunk']) for tok in tokens]
    input_tensor = torch.tensor(input_idxs).unsqueeze(0).to(device)

    # Start with BOS token
    generated = [word2idx['xxbos']]

    for _ in range(max_len):
        tgt_tensor = torch.tensor(generated).unsqueeze(0).to(device)  # (1, current_len)
        with torch.no_grad():
            output = model(input_tensor, tgt_tensor)  # call forward(src, tgt)
            next_token = output[0, -1].argmax().item()
            generated.append(next_token)

        if next_token == word2idx['xxeos']:
            break

    return idx_to_text(generated[1:], idx2word)  # skip BOS


from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def evaluate_model(df, model, tokenizer, word2idx, idx2word):
    scores = []
    for i, row in df.iterrows():
        input_text = row['text']
        true_summary = row['summary']
        pred_summary = generate_summary(input_text, model, tokenizer, word2idx, idx2word)
        rouge = scorer.score(true_summary, pred_summary)
        scores.append(rouge)

    # Average scores
    avg_rouge1 = np.mean([s['rouge1'].fmeasure for s in scores])
    avg_rougeL = np.mean([s['rougeL'].fmeasure for s in scores])
    print(f"\n✅ ROUGE-1 F1: {avg_rouge1:.4f}, ROUGE-L F1: {avg_rougeL:.4f}")

In [ ]:
import pickle  # Import the pickle module
# Load vocab again if needed
with open("thai_vocab.pkl", "rb") as f:
    word2idx = pickle.load(f)
idx2word = {i: w for w, i in word2idx.items()}

# Run a test inference
sample_input = " ด้วย ศูนย์ศึกษายุทธศาสตร์ สถาบันวิชาการป้องกันประเทศ กําหนดจัดการเสวนา แลกเปลี่ยนเรียนรู้ เรื่อง นักยุทธศาสตร์กับการพัฒนาประเทศ ในวันอังคารที่ 9 เมษายน พ.ศ. 2567 เวลา 0830-1030 ณ อาคารศูนย์นวัตกรรมการศึกษาทางทหาร สถาบันวิชาการป้องกันประเทศ รายละเอียด ตามสิ่งที่ส่งมาด้วย ในการนี้ ศูนย์ศึกษายุทธศาสตร์ฯ จึงขอเรียนเชิญผู้แทนหน่วย จํานวน 2 นาย เข้าร่วม การเสวนาแลกเปลี่ยนเรียนรู้ฯ ตามวัน และเวลาดังกล่าว ทั้งนี้ เพื่อให้การเตรียมการต้อนรับเป็นไปด้วย ความเรียบร้อย ขอความกรุณาส่งแบบตอบรับเข้าร่วมการเสวนาฯ 2 และ ขอขอบคุณมา ณ โอกาสนี้ "
print("\n📌 Input:", sample_input)
print("📄 Summary:", generate_summary(sample_input, model, tokenizer, word2idx, idx2word))

#Evaluate full dataset
evaluate_model(df, model, tokenizer, word2idx, idx2word)


📌 Input:  ด้วย ศูนย์ศึกษายุทธศาสตร์ สถาบันวิชาการป้องกันประเทศ กําหนดจัดการเสวนา แลกเปลี่ยนเรียนรู้ เรื่อง นักยุทธศาสตร์กับการพัฒนาประเทศ ในวันอังคารที่ 9 เมษายน พ.ศ. 2567 เวลา 0830-1030 ณ อาคารศูนย์นวัตกรรมการศึกษาทางทหาร สถาบันวิชาการป้องกันประเทศ รายละเอียด ตามสิ่งที่ส่งมาด้วย ในการนี้ ศูนย์ศึกษายุทธศาสตร์ฯ จึงขอเรียนเชิญผู้แทนหน่วย จํานวน 2 นาย เข้าร่วม การเสวนาแลกเปลี่ยนเรียนรู้ฯ ตามวัน และเวลาดังกล่าว ทั้งนี้ เพื่อให้การเตรียมการต้อนรับเป็นไปด้วย ความเรียบร้อย ขอความกรุณาส่งแบบตอบรับเข้าร่วมการเสวนาฯ 2 และ ขอขอบคุณมา ณ โอกาสนี้ 
📄 Summary: สศท . ส ปท. ขอ เชิญ ผู้แทน หน่วย ที่ สามารถ ให้ และ เข้าร่วม ได้ ประชุม ประชุม เร่งรัด กา รด ํา เนิน งาน และ เนิน งาน งาน ตาม นโยบาย ผบ. ท สส. ประ จํา ปี 2567 ใน วัน พฤหัสบดี ที่ พ.ค. 67 เวลา 0900 ณ ห้องประชุม บก . ทท . โดย มี ผบ. ท สส. เป็น ประธาน xxeos

✅ ROUGE-1 F1: 0.2400, ROUGE-L F1: 0.2400
